In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2021-04-16 14:08:42--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.26
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.26|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-04-16 14:08:42--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1594 (1.6K) [text/plain]
Saving to: ‘STDOUT’

-                     0%[                    ]       0  --.-KB/s               setup Colab for PySpark 3.0.2 and Spark NLP 3.0.1
-                   100%[====

In [ ]:
# from pyspark.sql import SparkSession as ss
# import pyspark.sql.functions as F
from pyspark.sql.types import StringType,StructType,StructField
# from pyspark.ml.linalg import VectorUDT, DenseVector
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd
import time
prog_start_time = time.time()
# import json
# sent_small_bert_L12_128
# spark = ss.builder.master("local[*]").getOrCreate()
spark = sparknlp.start() 
schema_sarc = StructType([StructField('u_id', StringType(), True),                     
                      StructField('text', StringType(), True),
                      StructField('sarcasm', StringType(), True)])


df_pan = pd.read_csv('/content/drive/MyDrive/sarcasm data/train_sarc_final.csv')

df = spark.createDataFrame(df_pan,schema_sarc)


# df = spark.read.option("header", True).option("inferSchema",True).csv("/Users/ishansharma/Desktop/NLP/Model Selection/Imbalanced problem/train_sent")
# df = df.withColumnRenamed("preprocessed","text")
document = DocumentAssembler().setInputCol("text").setOutputCol("document")
# use = UniversalSentenceEncoder.pretrained().setInputCols(["document"]).setOutputCol("USE_pre_text_all_upd_embed")
tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("token")
spellModel = ContextSpellCheckerModel.pretrained().setInputCols("token").setOutputCol("spell")
# lemmatizer = LemmatizerModel.pretrained(name="lemma_antbnc").setInputCols(["spell"]).setOutputCol("lemma").setLazyAnnotator(False)
sentenceDetector = SentenceDetector().setInputCols(["document"]).setOutputCol("sentence")
embeddings = BertEmbeddings.pretrained("small_bert_L12_128", "en").setInputCols("sentence", "spell").setOutputCol("embeddings")
embeddings_finisher = EmbeddingsFinisher().setInputCols("embeddings").setOutputCols("embeddings_vectors").setOutputAsVector(True)
# # bert = BertSentenceEmbeddings.pretrained("sent_electra_small_uncased", "en").setInputCols("sentence").setOutputCol("sentence_embeddings")
# # # use_clf_pipeline = Pipeline(
# #     stages = [
# #         document,
# #         use
# #     ])

lemm_pipeline = Pipeline(
    stages = [
        document,
        tokenizer,
        spellModel,
        # lemmatizer,
        sentenceDetector,
        embeddings,
        embeddings_finisher
        # spell_checker,
#         lemmatizer
    ])


df_lem = lemm_pipeline.fit(df).transform(df)
print("TRANSFORMATIONS DONEEEEE!!!!! \n")
# df_lem.printSchema()
# ud_f = F.udf(lambda r : DenseVector(r),F.StringType())
# res_df = df_bert.select("id",F.explode("sentence_embeddings").alias("embeddings"),"sarcasm","final_sent_class")
# res_df = res_df.select("id","embeddings.embeddings","sarcasm","final_sent_class")
# res_df = res_df.withColumn("BERT_embed",ud_f(F.col("embeddings")))
# res_df = res_df.drop("embeddings")
# print("SAVING !!!!")
# conv_start_time = time.time()
# df_lem.select("u_id","embeddings_vectors","sentiment").toPandas().to_csv('/Users/ishansharma/Desktop/NLP/Model Selection/Imbalanced problem/train_sent_elec_embed/test.csv',index = False,header = True)
# print("CONVERSION END TIME: %d, PROGRAM END TIME: %d"%(time.time()-conv_start_time,time.time()-prog_start_time))

spellcheck_dl download started this may take some time.
Approximate size to download 111.4 MB
[OK!]
small_bert_L12_128 download started this may take some time.
Approximate size to download 23.4 MB
[OK!]
TRANSFORMATIONS DONEEEEE!!!!! 



In [ ]:
df_lem = df_lem.select("u_id","embeddings_vectors","sarcasm")

In [ ]:
df_fin = df_lem.withColumn("embeddings_vectors",df_lem.embeddings_vectors.cast('string'))

In [ ]:
df_fin.coalesce(1).write.option("header","true").csv('/content/drive/MyDrive/sarcasm data/sarcasm_embed_final')

KeyboardInterrupt: ignored